In [ ]:
#author Kim Iheon
#verson 3
#적합도 구하는 방법 수정
#불필요한 매개변수 gene_list 삭제
#룰렛휠 선택을 이용한 선택 연산 제작

import random


#유전자를 생성하고 진화시키는 클래스
class Gene:
    def init(self):
        pass
    
    #첫번째 세대의 유전자를 생성하는 함수
    #param {Int} num_net 개체-신경망-의 개수
    #param {Int} num_node 한 신경망의 초기 노드 수
    #param {Int} num_route 한 신경망에서 초기 시냅스 수
    #param {Int} num_in 입력 노드의 개수
    #param {Int} num_out 출력 노드의 개수
    #return {List} gene_list 첫번째 세대의 유전자를 리턴
    def create_gene(self, num_net, num_node, num_sys, num_in, num_out):
        self.NUM_NET = num_net
        self.NUM_NODE = num_node
        self.NUM_SYS = num_sys
        self.NUM_IN = num_in
        self.NUM_OUT = num_out
        self.gene_list = []
        
        #현재 세대 수
        self.generation = 1
        
        #하나의 개체
        _network = []
        
        #한 세대의 모든 유전자
        gene_list = []
        
        #존재하는 모든 노드의 리스트
        _node_list = list(range(num_node))
        
        #시냅스의 시작과 끝에 존재가능한 노드
        _start_list = _node_list[:num_node - num_out]
        _end_list = _node_list[num_in:]
        
        for m in range(num_net):
            for n in range(num_node):
                _start_node = random.choice(_start_list)
                
                #시작과 끝이 중복일 가능성을 제거
                if _start_node in _end_list:
                    del _end_list[_start_node]
                
                _end_node = random.choice(_end_list)
                _end_list = _node_list[num_in:]
                synapse = (_start_node, _end_node)
                
                #시냅스의 연결 여부
                connect = random.randint(0, 1)
                _network.append((synapse, connect))
            gene_list.append(_network)
            
        self.gene_list = gene_list
        
        return gene_list
    
    #return {List} self.gene_list 현재 세대의 유전자를 리턴
    def get_gene(self):
        return self.gene_list
    
    #현재 세대 모든 개체의 적합도 리스트를 리턴
    #param {Function} qual_func 개체의 품질을 정하는 함수
    #return {List} fit_list 현재 세대의 모든 개체의 적합도
    def fitness(qual_func):
        
        #현재 세대의 모든 개체의 적합도
        self_fit_list = []
        
        #현재 새대의 모든 개채를 받아옴
        gene_list = self.gene_list
        
        #품질의 최고값과 최저값
        MAX_QUAL = 1
        MIN_QUAL = 0
        
        #적합도 리스트
        fit_list = []
        
        #가장 좋은 품질과 가장 안 좋은 품질
        best_qual = MIN_QUAL
        werst_qual = MAX_QUAL
        
        for i in range(self.NUM_NET):
            i_qual = qual_func(gene_list[i])
            if best_qual < i_qual:
                best_qual = i_qual
            if werst_qual > qual_func(gene_list[i]):
                werst_qual = i_qual
        
        #각 개체에 대해 적합도를 계산
        for i in range(self.NUM_NET):
            i_qual = qual_func(gene_list[i])
            i_fit = (werst_qual - i_qual) \
                    + ((werst_qual - best_qual) / (k - 1))
            fit_list.append(i_fit)
        
        self.fit_list = fit_list
        
        return fit_list
    
    #적합도 리스트와 룰렛 휠 선택을 이용해 개체를 선택
    #param {Tuple} num 선택하는 개체의 개수. 없으면 1개 선택
    #return {List} sel_list 선택된 개체의 번호 리스트
    def selection(self, *num):
        sel_list = []
        
        #현재 새대의 모든 개체를 받아옴
        gene_list = self.gene_list
        
        #이전에 fitness를 실행했어야함
        fit_list = self.fit_list
        
        #선택하는 개체수. 입력하지 않으면 1개 선택
        cycle = 1 if num == () else num[0]
        point = random.uniform(0, sum(fitness(gene_list)))
        sum_fit = 0
        
        for k in range(cycle):
            for i in range(self.NUM_NET):
                sum_fit += fit_list[i]
                if point < sum_fit:
                    sel_list.append(i)
                    break
        
        return sel_list
    
    def crossover(self, sel_list):
        pass
    
    def mutation(self):
        pass
    
    def replacement(self):
        pass
    
    def evolution(self, qual_func):
        self.generation += 1
        pass